Experimenting with the Convolutional Neural Networks and Tensorflow following this YouTube course https://www.youtube.com/watch?v=qFJeN9V1ZsI sponsored or created by https://deeplizard.com/ This is definitely a very good introduction course.

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

In [3]:
# Organize data into train, valid, test dirs
os.chdir('data/dogs-vs-cats')
if os.path.isdir('train/dog') is False:
    os.makedirs('train/dog')
    os.makedirs('train/cat')
    os.makedirs('valid/dog')
    os.makedirs('valid/cat')
    os.makedirs('test/dog')
    os.makedirs('test/cat')

    for i in random.sample(glob.glob('cat*'), 500):
        shutil.move(i, 'train/cat')      
    for i in random.sample(glob.glob('dog*'), 500):
        shutil.move(i, 'train/dog')
    for i in random.sample(glob.glob('cat*'), 100):
        shutil.move(i, 'valid/cat')        
    for i in random.sample(glob.glob('dog*'), 100):
        shutil.move(i, 'valid/dog')
    for i in random.sample(glob.glob('cat*'), 50):
        shutil.move(i, 'test/cat')      
    for i in random.sample(glob.glob('dog*'), 50):
        shutil.move(i, 'test/dog')

os.chdir('../../')

In [5]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory='data/dogs-vs-cats/train', target_size=(224,224), classes=['cat', 'dog'], batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory='data/dogs-vs-cats/valid', target_size=(224,224), classes=['cat', 'dog'], batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory='data/dogs-vs-cats/test', target_size=(224,224), classes=['cat', 'dog'], batch_size=10, shuffle=False)

Found 1000 images belonging to 2 classes.
Found 200 images belonging to 2 classes.
Found 100 images belonging to 2 classes.


In [13]:
model = Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same', input_shape=(224,224,3)),
    MaxPool2D(pool_size=(2,2), strides=2),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(pool_size=(2,2), strides=2),
    Flatten(),
    Dense(units=2, activation='softmax'),
])

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 112, 112, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 200704)            0         
_________________________________________________________________
dense (Dense)                (None, 2)                 401410    
Total params: 420,802
Trainable params: 420,802
Non-trainable params: 0
__________________________________________________

In [19]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [20]:
model.fit(x=train_batches, validation_data=valid_batches, epochs=10, verbose=2)

Epoch 1/10
100/100 - 20s - loss: 16.0408 - accuracy: 0.5560 - val_loss: 5.9844 - val_accuracy: 0.6050
Epoch 2/10
100/100 - 21s - loss: 2.9080 - accuracy: 0.7480 - val_loss: 3.9506 - val_accuracy: 0.6350
Epoch 3/10
100/100 - 21s - loss: 0.8525 - accuracy: 0.8800 - val_loss: 3.0100 - val_accuracy: 0.6300
Epoch 4/10
100/100 - 20s - loss: 0.1460 - accuracy: 0.9640 - val_loss: 2.8969 - val_accuracy: 0.6450
Epoch 5/10
100/100 - 20s - loss: 0.0503 - accuracy: 0.9840 - val_loss: 2.9282 - val_accuracy: 0.6500
Epoch 6/10
100/100 - 21s - loss: 0.0982 - accuracy: 0.9710 - val_loss: 3.1263 - val_accuracy: 0.6650
Epoch 7/10
100/100 - 21s - loss: 0.0714 - accuracy: 0.9810 - val_loss: 2.6378 - val_accuracy: 0.6350
Epoch 8/10
100/100 - 22s - loss: 0.0364 - accuracy: 0.9910 - val_loss: 2.7545 - val_accuracy: 0.6300
Epoch 9/10
100/100 - 21s - loss: 0.0014 - accuracy: 1.0000 - val_loss: 2.4665 - val_accuracy: 0.6600
Epoch 10/10
100/100 - 23s - loss: 4.1077e-04 - accuracy: 1.0000 - val_loss: 2.4189 - val_a